In [ ]:
# ============================================================================
# EASY TECHNICAL SIGNAL SCANNER - USER-FRIENDLY INTERFACE
# ============================================================================
# This is a simplified wrapper around the TechnicalAnalyzer class
# Allows users to easily run analysis with minimal code
# ============================================================================

import os
import sys
from datetime import datetime
from pathlib import Path

# Import the main analyzer (assumes technical_analyzer.py in same directory)
try:
    from technical_analyzer import TechnicalAnalyzer, create_signals_dataframe
    from technical_analyzer import get_category_breakdown, get_strength_analysis, export_to_csv
except ImportError:
    print("❌ Error: Could not import technical_analyzer.py")
    print("   Make sure technical_analyzer.py is in the same directory")
    sys.exit(1)


class EasySignalScanner:
    """
    Simplified interface for running technical analysis on any stock ticker
    and any time period
    """
    
    # Valid time periods
    VALID_PERIODS = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'max']
    
    def __init__(self, symbol=None, period=None, gemini_api_key=None):
        """
        Initialize the scanner
        
        Args:
            symbol: Stock ticker (e.g., 'AAPL', 'TSLA')
            period: Time period ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'max')
            gemini_api_key: Optional Gemini API key for AI signal ranking
        """
        self.symbol = symbol
        self.period = period
        self.gemini_api_key = gemini_api_key or os.getenv('GEMINI_API_KEY')
        self.analyzer = None
    
    def display_welcome(self):
        """Display welcome banner"""
        print("\n" + "="*80)
        print("  📊 EASY TECHNICAL SIGNAL SCANNER")
        print("  Advanced Technical Analysis with 150+ Signals & Fibonacci Levels")
        print("="*80 + "\n")
    
    def get_user_input(self):
        """Get stock ticker and period from user if not provided"""
        if not self.symbol:
            print("Enter stock ticker (e.g., AAPL, TSLA, NVDA):")
            self.symbol = input(">>> ").strip().upper()
            
            if not self.symbol:
                print("❌ Ticker cannot be empty")
                return False
        
        if not self.period:
            print(f"\nAvailable time periods: {', '.join(self.VALID_PERIODS)}")
            print("Enter time period (default: 1y):")
            period_input = input(">>> ").strip().lower() or '1y'
            
            if period_input not in self.VALID_PERIODS:
                print(f"❌ Invalid period. Use one of: {', '.join(self.VALID_PERIODS)}")
                return False
            
            self.period = period_input
        
        return True
    
    def run_analysis(self, verbose=True):
        """
        Run the complete analysis
        
        Args:
            verbose: Print detailed output (default: True)
        
        Returns:
            TechnicalAnalyzer object with all results
        """
        if not self.symbol or not self.period:
            if not self.get_user_input():
                return None
        
        try:
            if verbose:
                self.display_welcome()
                print(f"🚀 Starting analysis for {self.symbol} ({self.period})")
                if self.gemini_api_key:
                    print("   ✅ AI ranking enabled (Gemini API)")
                else:
                    print("   ℹ️  AI ranking disabled (set GEMINI_API_KEY to enable)")
                print()
            
            # Create analyzer
            self.analyzer = TechnicalAnalyzer(
                symbol=self.symbol,
                period=self.period,
                gemini_api_key=self.gemini_api_key
            )
            
            # Run analysis pipeline
            self.analyzer.fetch_data()
            self.analyzer.calculate_indicators()
            self.analyzer.detect_signals()
            self.analyzer.rank_signals_with_ai()
            self.analyzer.save_locally()
            
            if verbose:
                self.analyzer.display_results()
            
            return self.analyzer
        
        except Exception as e:
            print(f"❌ Analysis Error: {str(e)}")
            return None
    
    def get_top_signals(self, count=10):
        """Get top N signals"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return self.analyzer.signals[:count]
    
    def get_signals_dataframe(self):
        """Get signals as pandas DataFrame"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return create_signals_dataframe(self.analyzer)
    
    def export_csv(self, filename=None):
        """Export signals to CSV"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return export_to_csv(self.analyzer, filename)
    
    def export_json(self, filename=None):
        """Export signals to JSON"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        import json
        if not filename:
            filename = f"{self.symbol}_signals_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        filepath = os.path.join(self.analyzer.date_folder, filename)
        
        signals_data = {
            'symbol': self.symbol,
            'period': self.period,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'total_signals': len(self.analyzer.signals),
            'signals': self.analyzer.signals
        }
        
        with open(filepath, 'w') as f:
            json.dump(signals_data, f, indent=2)
        
        print(f"\n✅ Exported to: {filepath}")
        return filepath
    
    def filter_by_category(self, category):
        """Filter signals by category"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return [s for s in self.analyzer.signals if s['category'] == category]
    
    def filter_by_score(self, min_score=70):
        """Filter signals by minimum AI score"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return [s for s in self.analyzer.signals 
                if s.get('ai_score', 0) >= min_score]
    
    def get_fibonacci_signals(self):
        """Get only Fibonacci signals"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return [s for s in self.analyzer.signals 
                if 'FIBONACCI' in s['category']]
    
    def get_ma_signals(self):
        """Get only Moving Average signals"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return None
        
        return [s for s in self.analyzer.signals 
                if 'MA' in s['category']]
    
    def print_summary(self):
        """Print analysis summary"""
        if not self.analyzer:
            print("❌ No analysis data. Run analysis first.")
            return
        
        current = self.analyzer.data.iloc[-1]
        
        bullish = sum(1 for s in self.analyzer.signals if 'BULLISH' in s['strength'])
        bearish = sum(1 for s in self.analyzer.signals if 'BEARISH' in s['strength'])
        
        print("\n" + "="*80)
        print("📊 ANALYSIS SUMMARY")
        print("="*80)
        print(f"\nSymbol: {self.symbol} | Period: {self.period}")
        print(f"Price: ${current['Close']:.2f} | Change: {current['Price_Change']:+.2f}%")
        print(f"Date: {current.name.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"\n📈 Technical Indicators:")
        print(f"   RSI: {current['RSI']:.1f} | MACD: {current['MACD']:.4f} | ADX: {current['ADX']:.1f}")
        print(f"   Volatility: {current['Volatility']:.1f}% | ATR: {current['ATR']:.2f}")
        print(f"\n🎯 Signal Overview:")
        print(f"   Total Signals: {len(self.analyzer.signals)}")
        print(f"   Bullish: {bullish} | Bearish: {bearish}")
        bias = "🟢 BULLISH" if bullish > bearish else "🔴 BEARISH" if bearish > bullish else "🟡 NEUTRAL"
        print(f"   Bias: {bias}")
        print(f"\n📂 Results saved to: {self.analyzer.date_folder}")
        print("="*80 + "\n")


# ============================================================================
# QUICK START FUNCTIONS
# ============================================================================

def scan(symbol, period='1y', gemini_key=None):
    """
    Quick scan function - minimal code
    
    Usage:
        analyzer = scan('AAPL', '1y')
        analyzer = scan('TSLA', '6mo', os.getenv('GEMINI_API_KEY'))
    """
    scanner = EasySignalScanner(symbol, period, gemini_key)
    return scanner.run_analysis()


def scan_interactive():
    """
    Interactive mode - prompts for inputs
    
    Usage:
        analyzer = scan_interactive()
    """
    scanner = EasySignalScanner()
    scanner.display_welcome()
    if scanner.get_user_input():
        return scanner.run_analysis()
    return None


# ============================================================================
# EXAMPLE USAGE - UNCOMMENT TO RUN
# ============================================================================

if __name__ == "__main__":
    
    # ========== OPTION 1: INTERACTIVE MODE (Easiest) ==========
    # Prompts user for stock ticker and period
    # Uncomment this to use:
    
    analyzer = scan_interactive()
    
    
    # ========== OPTION 2: DIRECT MODE (Quickest) ==========
    # Specify everything directly
    # Uncomment to use instead of Option 1:
    
    # import os
    # analyzer = scan('AAPL', '1y', os.getenv('GEMINI_API_KEY'))
    
    
    # ========== OPTION 3: ADVANCED MODE (Most Control) ==========
    # Use the full EasySignalScanner class
    # Uncomment to use instead of Option 1:
    
    # import os
    # scanner = EasySignalScanner(symbol='TSLA', period='6mo', 
    #                             gemini_api_key=os.getenv('GEMINI_API_KEY'))
    # analyzer = scanner.run_analysis()
    # 
    # # Now you can:
    # signals_df = scanner.get_signals_dataframe()
    # print(signals_df.head(10))
    # 
    # fib_signals = scanner.get_fibonacci_signals()
    # print(f"Fibonacci signals: {len(fib_signals)}")
    # 
    # high_confidence = scanner.filter_by_score(75)
    # print(f"High confidence signals: {len(high_confidence)}")
    # 
    # scanner.export_csv()
    # scanner.export_json()
    
    
    if analyzer:
        # ========== POST-ANALYSIS OPTIONS ==========
        
        # Get top 10 signals
        # top_signals = analyzer.signals[:10]
        # for i, sig in enumerate(top_signals, 1):
        #     print(f"{i}. {sig['signal']} (Score: {sig.get('ai_score', 'N/A')})")
        
        # Get Fibonacci signals only
        # fib_sigs = [s for s in analyzer.signals if 'FIBONACCI' in s['category']]
        # print(f"Fibonacci signals: {len(fib_sigs)}")
        
        # Export results
        # signals_df = create_signals_dataframe(analyzer)
        # signals_df.to_csv(f"{analyzer.symbol}_signals.csv", index=False)
        # print("✅ Exported to CSV")
        
        print("\n✅ Analysis complete!")
        print("\nYou can now access:")
        print("  - analyzer.signals         : List of all detected signals")
        print("  - analyzer.data            : Technical indicator data")
        print("  - analyzer.date_folder     : Location of saved files")